## KnowEnGSpreadsheetOrientedObject:   KSOO v00
    * Introspective Spreadsheet in the context of KnowEnG Pipeline Anaylsis
    * HBox Directory - File Selector
    
# Stop Crashing this computer and call:
[os.walk(),...](https://stackoverflow.com/questions/10989005/do-i-understand-os-walk-right)

```python
# just get the directories with files list:
import os.path

for path, directories, files in os.walk('C:\dir1\dir2\startdir'):
     if file in files:
          print 'found %s' % os.path.join(path, file)
```

In [111]:
import os
from os.path import expanduser

from IPython.display import display
from ipywidgets import widgets

file_select_box_layout = widgets.Layout(display='inline-flex', 
                                        flex_flow='row',
                                        justify_content='space-between', 
                                        align_items='stretch',
                                        border='none', width='100%')

PARAMETER_FILE_TYPES = ['.yml']
USER_DATAFILE_EXTENSIONS_LIST = ['.tsv', '.txt', '.df', '.gz', '.csv']
ALL_DATA_FILE_TYPES = PARAMETER_FILE_TYPES + USER_DATAFILE_EXTENSIONS_LIST
NO_FILES_LIST = ['No Data']
DIR_IGNORE = ['Google Drive', 'Accounts']

PREFIX_IGNORE = ['.', ' ', '_']

MAX_DIR_RECURSION = 2

def get_relevant_data_directories(target_dir, FEXT=ALL_DATA_FILE_TYPES):
    dir_dict = {}
    for path, directories, files in os.walk(target_dir):
        _, path_0 = os.path.split(path)
        if len(path_0) < 1 or path_0[0] in PREFIX_IGNORE: 
            continue

        for file_name in files:
            _, ext_ = os.path.splitext(file_name)
            if ext_ in FEXT:
                dir_dict[path_0] = path
                break
                
    return dir_dict

def user_data_list(target_dir, FEXT):
    """ user_file_list = update_user_data_list(user_data_dir, FEXT)
    Args:
        target_dir:     directory to list
        FEXT:           File extension list e.g. ['.tsv', '.txt']
    """
    my_file_list = []
    if os.path.isdir(target_dir) == False:
        return my_file_list
    
    for f in os.listdir(target_dir):
        if os.path.isfile(os.path.join(target_dir, f)):
            noNeed, f_ext = os.path.splitext(f)
            if f_ext in FEXT:
                my_file_list.append(f)
            else:
                pass

    if len(my_file_list) <= 0:
        my_file_list.append('No Data')

    return my_file_list


class HorizontalFileSelector():
    def __init__(self, user_home_dir=None):
        if user_home_dir is None or os.path.isdir(os.path.abspath(user_home_dir)) == False:
            self._user_data_directory = expanduser('~')
        else:
            self._user_data_directory = os.path.abspath(user_home_dir)
        
        #                             Create the set of controls in a horizontal Box
        self.directory_valid = widgets.Valid(value=False, description='directory')
        self.directory_dropd = widgets.Dropdown(options=NO_FILES_LIST, value=NO_FILES_LIST[0], description='', disabled=True)

        self.filename_valid  = widgets.Valid(value=False, description='file')
        self.filename_dropd  = widgets.Dropdown(options=NO_FILES_LIST, value=NO_FILES_LIST[0], description='', disabled=True)

        self.file_select_set = widgets.HBox([self.directory_valid, 
                                             self.directory_dropd,
                                             self.filename_valid, 
                                             self.filename_dropd], 
                                            layout=file_select_box_layout)
        self.set_directory_dropd()
        self.set_filename_dropd('null')
        
        self.directory_dropd.observe(self.set_filename_dropd, 'value')
        
    def show(self):
        display(self.file_select_set)
        
    def get_directory_dropd(self):
        return os.path.join(self._user_data_directory, self.directory_dropd.value)
        
    def set_directory_dropd(self):
        dropd_dir_dict = get_relevant_data_directories( self._user_data_directory )
        if len(dropd_dir_dict) > 0:
            dropd_dir_list = sorted(list(dropd_dir_dict.keys()))
            self.directory_dropd.options = dropd_dir_list
            self.directory_dropd.value   = dropd_dir_list[0]
            selected_directory = self.get_directory_dropd()
            
            if os.path.isdir(selected_directory):
                self.set_filename_dropd('null')
            self.directory_dropd.disabled = False
            self.directory_valid.value = True
        else:
            self.directory_dropd.options = ['none']
            self.directory_dropd.value   = 'none'
            self.directory_dropd.disabled = True
            self.directory_valid.value = False

    def set_filename_dropd(self, _):
        selected_dir = os.path.join(self._user_data_directory, self.directory_dropd.value)
        options_list = user_data_list(selected_dir, USER_DATAFILE_EXTENSIONS_LIST)
        if len(options_list) < 1:
            options_list = NO_FILES_LIST
        self.filename_dropd.options = options_list
        self.filename_dropd.value = options_list[0]
        self.filename_dropd.disabled = False
        self.filename_valid.value = True
        

In [113]:
NO_FILES_LIST[0]

'No Data'

In [114]:
home = '/Users/mojo/git_clone/dlanier/'
# os.path.isdir(home)
hfs = HorizontalFileSelector('/Users/mojo/git_clone/dlanier/pipe_tools')
hfs.show()



HBox(children=(Valid(value=True, description='directory'), Dropdown(options=('notebooks', 'versions'), value='notebooks'), Valid(value=True, description='file'), Dropdown(options=('test_II_versions.txt', 'test_versions.txt', 'installed_packages.txt'), value='test_II_versions.txt')), layout=Layout(align_items='stretch', border='none', display='inline-flex', flex_flow='row', justify_content='space-between', width='100%'))